In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import yellowbrick 
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Concatenate
import csv
stroke = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
stroke.info()

info metodu ile stroke data setinde null olan alanlar ve alan tipleri hakkında bilgi ediniriz.İnteger(4 adet), string(5 adet) ve float(3 adet) olan veri tipleri mevcur.bmi kolonunda null (201 adet)değerler var.11 kolondan oluşan bir veri seti ile çalışacağız.

1) id: unique identifier
2) gender: "Male", "Female" or "Other"
3) age: age of the patient
4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6) ever_married: "No" or "Yes"
7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8) Residence_type: "Rural" or "Urban"
9) avg_glucose_level: average glucose level in blood
10) bmi: body mass index
11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
12) stroke: 1 if the patient had a stroke or 0 if not
*Note: "Unknown" in smoking_status means that the information is unavailable for this patient

In [ ]:
stroke.describe()

"describe" komutu ile veri setinin istatistiksel(adet, ortalama, standart sapma, kümedeki minimum ve maksimum değerler ,percantiller) bilgiler hakkında bilgi ediniriz.5110 satır veri var.Yaş ortalaması 43,22.Maksimum yaş 82.

In [ ]:
stroke.head()

Veri setinin ilk 5 satırındaki veriler.

In [ ]:
stroke.tail()

Veri setinin en son 5 satırdaki(kuyruktaki)verileri.

In [ ]:
print(stroke['gender'][stroke['heart_disease']==1].value_counts())

Veriye göre kalp hastalığı olanların 163'ü erkek,113'ü bayanmış.Erkeklerde kalp hastalığı daha fazla görülmüş.

In [ ]:
print(stroke['gender'][stroke['hypertension']==1].value_counts())

Veri setinde hiper tansiyonu olanların 276 'sı bayan, 222 'si erkekmiş.Bayanlarda hipertansiyon daha fazla görülmüş.

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Age & Avg_Glucose_Level')
for gender in ['Male', 'Female']:
    plt.scatter(stroke[stroke.gender == gender].age, 
                stroke[stroke.gender == gender].avg_glucose_level, label = gender)
plt.xlabel('age')
plt.ylabel('avg_glucose_level')
plt.legend()
plt.show()

Bayan ve erkeklerin yaşlarına göre şeker hastalığı düzeyleri.40-80 yaşları arası hem erkeklerde hem kadınlarda kandaki şeker seviyesi artıyor.

In [ ]:
plt.figure(figsize=(5, 3))
x = np.random.normal(43, 23, 5110)
plt.hist(x, color="#22B573") 
plt.show()

Veri setindeki yaş grupları 25 ile 55 arasında yoğun.

In [ ]:
stroke.isnull()

"bmi" sutununda boş değerler var.

In [ ]:
stroke.nunique()

tekil verileri gösterir.Bu sayede hangi sutunların kategorik olduğunu anlarız.Burada yaş, glokoz düzeyi ve bmi sürekli değişkenlerdir.

In [ ]:
stroke["bmi"].fillna(stroke["bmi"].mean(), inplace=True)

bmi sutunundaki eksik değerleri ortalaması ile doldurduk.

In [ ]:
stroke.info()

Tekrar info metodu ile bmi sutunundaki eksik değerlerin doldugunu ,5110 adet satır veri içerdiğini gözlemledik.

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Bmi & Avg_Glucose_Level')
for gender in ['Male', 'Female']:
    plt.scatter(stroke[stroke.gender == gender].bmi, 
                stroke[stroke.gender == gender].avg_glucose_level, label = gender)
plt.xlabel('bmi')
plt.ylabel('avg_glucose_level')
plt.legend()
plt.show()

Bmi düşükken bayan ve erkeklerde kandaki şeker düzeyi de 50-150 arasında yoğunlaşmış.bmi 40-60 arasındayken kandaki şeker düzeyi 0-150 arasında 150-250 arasına göre bir miktar daha yoğun .Düşük bmi kandaki şeker düzeyi 0-150 arasında olan bayanlar erkeklere göre daha fazla.

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1,3,1)
plt.title('Age')
ax = plt.boxplot(stroke['age'].dropna())
plt.xlabel('Age')
plt.ylabel('Stroke')

plt.subplot(1,3,2)
plt.title('avg_glucose_level')
ax = plt.boxplot(stroke['avg_glucose_level'].dropna())
plt.xlabel('Avg_glucose_level')
plt.ylabel('Stroke')

plt.subplot(1,3,3)
plt.title('bmi')
ax = plt.boxplot(stroke['bmi'].dropna())
plt.xlabel('Bmi')
plt.ylabel('Stroke')

plt.show()

Aykırı değerleri gözlemlemek için boxplot grafiği çizdirdik. Çalışmaya başlamadan önce veri setinde aykırı değerleri kontrol etmeliyiz.Varsa, bu aykırı değerleri limitlememiz ya da etkisini azaltmamız gerekli.Bıyıkların dışındaki (üst ve alt yatay çizgiler)noktalar aykırı değerlerdir.

In [ ]:
from scipy.stats.mstats import winsorize
y=winsorize(stroke['bmi'],(0,0.15))
y

In [ ]:
y=winsorize(stroke['avg_glucose_level'],(0,0.15))
y

In [ ]:
stroke.corr()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(stroke.corr(), annot=True, fmt='.2f',annot_kws={"size":10})
plt.show()

In [ ]:
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X = stroke.drop(['id', 'stroke'], axis=1)
X = pd.get_dummies(X, drop_first=True)
y = stroke['stroke']

In [ ]:
stroke_dmatrix = xgb.DMatrix(data=X, label=y)

In [ ]:
params = {"objective":"reg:logistic", "max_depth":3}
stroke_cv = xgb.cv(dtrain=stroke_dmatrix, params=params, nfold=3, 
                    num_boost_round=4, metrics="error", as_pandas=True, seed=123)
display(stroke_cv)

In [ ]:
print('Doğruluk Değeri : {:.3f}'.format((1-stroke_cv["test-error-mean"]).max()))

In [ ]:
stroke_auc = xgb.cv(dtrain=stroke_dmatrix, params=params, nfold=3, 
                    num_boost_round=4, metrics="auc", as_pandas=True, seed=123)
display(stroke_auc)
print('AUC Skoru : {:.3f}'.format(stroke_auc["test-auc-mean"].max()))

XGBoost algoritması ile denedikten sonra şimdi Light GBM algoritması ile deneyelim.Doğruluk ve AUC değeri 1'e yakın olduğu için iyi görünüyor.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

verilerimizi %20 test, %80 eğitim verisi olacak şekilde böldük.

In [ ]:
import lightgbm as lgb

Light gbm kütüphanesini lgb olarak indirdik.

In [ ]:
d_train = lgb.Dataset(X_train, label=y_train)

Verileri Lgb veri formatına çevirdik.

In [ ]:
params = {'boosting_type' : 'gbdt',
          'objective' : 'binary',
          'metric' : 'binary_logloss',
          'sub_feature' : 0.5,
          'num_leaves' :  10,
          'min_data' : 50,
          'max_depth' : 10,
          'force_row_wise':True,
          'verbose': -1
          ,
         }

Modelimizde kullanacağımız parametreleri belirledik.

In [ ]:
lgb_model = lgb.train(params, d_train, num_boost_round = 100, verbose_eval=False)

Kullanacağımız parametrelerle modelimizi tanımladık.Modelimizi eğittik.

In [ ]:
y_tahmin=lgb_model.predict(X_test)
y_tahmin[:10]

Tahmin sonucu olarak olasılık değerleri verir .Eşik değer vererek bunları tahminlere çeviririz.

In [ ]:
y_tahmin = [0 if tahmin < 0.5 else 1 for tahmin in y_tahmin]
y_tahmin[:10]

In [ ]:
from sklearn.metrics import accuracy_score
dogruluk=accuracy_score(y_tahmin,y_test)
dogruluk

Modelimizin doğruluk skoruna baktık.1'e yakın olduğu için oldukça iyi sonuç verdi.XGBoost ile LightGBM doğruluk değerleri oldukça yakın.XGBoost algoritması bir miktar daha iyi.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN

#Standardizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

Kmeans ,Mini BatchKMeans ,PCA ve Standardscaler kütüphanelerini indirdik.Verimizi ölçeklendirdik.

In [ ]:
# Defining the k-means
kmeans_küme = KMeans(n_clusters=2, random_state=123)

# Fit model
%timeit kmeans_küme.fit(X_std)
y_pred = kmeans_küme.predict(X_std)

3 küme olacağı için cluster sayısını 3 tanımladık.Zamanı ölçerek modelimizi eğittik.Tahminlerimizi yaptık.

In [ ]:
pca = PCA(n_components=2).fit_transform(X_std)

plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(pca.shape[0]):
    plt.text(pca[i, 0], pca[i, 1], str(y_pred[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

Ikı boyutlu uzayda görselleştirmek için PCA metodunu kullandık.

In [ ]:
# Defining the mini-batch k-means
minikmeans_cluster = MiniBatchKMeans(
    init='random',
    n_clusters=2,
    batch_size=30)

# Fit model
%timeit minikmeans_cluster.fit(X_std)
minikmeans_cluster = minikmeans_cluster.predict(X_std)

plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(pca.shape[0]):
    plt.text(pca[i, 0], pca[i, 1], str(minikmeans_cluster[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

MiniBatch uygulayarak  30 'lu batch'ler olarak veriyi proses ettik.PCA ile görselleştirdik. Modelimiz hızlandı.

In [ ]:
pd.crosstab(y, y_pred) 

0. sınıfa ait 247 gözlemi 1. sınıfa ,1. sınıfa ait 710 gözlemi 0. sınıfa atmış.Bu değerler yanlış sınıflara atanmış.4151 adet 0 olan sınıf doğru tahminlenmiş.2 adet 1 olan sınıf doğru şekilde tahmin edilmiş.

In [ ]:
metrics.adjusted_rand_score(y, y_pred)

ARI metriğine göre tahmin performansı kötü görünüyor.Şimdi değer kümeleme yöntemi olan DBSCAN ile bakalım.

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

Ölçeklendirme uyguladık.

In [ ]:
dbscan_cluster = DBSCAN(eps=1, min_samples=5)

clusters = dbscan_cluster.fit_predict(X_std)

Parametreleri belirleyip, modeli eğittik.

In [ ]:
pca = PCA(n_components=2).fit_transform(X_std)

plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(pca.shape[0]):
    plt.text(pca[i, 0], pca[i, 1], str(clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

PCA ile modeli uzayda görselleştirdik.

In [ ]:
print("DBSCAN çözümünün Düzeltilmiş Rand Endeksi (ARI): {}"
      .format(metrics.adjusted_rand_score(y, clusters)))

ARI ve siluet metriklerine bakarak model performansını değerlendirdik.

In [ ]:
print("DBSCAN çözümünün siluet skoru: {}"
      .format(metrics.silhouette_score(X_std, clusters, metric='euclidean')))

Performansı kötü görünüyor.-1 olarak etiketlenenler tüm veriler içerisinde gürültü verileridir.

Derin öğrenme algoritması olan RNN ile modeli eğitip tahminlerimizi yapalım.

In [ ]:
import yellowbrick
import pandas as pd
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Concatenate
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
import warnings
warnings.filterwarnings("ignore")
title_font= {"family": "arial", "weight": "bold", "color": "darkred", "size":13}
label_font= {"family": "arial", "weight": "bold", "color": "darkblue", "size":10}

Tensorflow.keras içerisindeki Dense,LSTM kütüphanesini indirdik.

Eğitim ve test verilerini %70, %30 oranında bölelim.Shape'e bakalım.

In [ ]:
X = stroke.drop(['id', 'stroke'], axis=1)
X = pd.get_dummies(X, drop_first=True)
y = stroke['stroke']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.3, random_state=123)

print("Train set shape (X) {}".format(X_train.shape))
print("Train set shape (y) {}".format(y_train.shape))
print("Test set shape (X) {}".format(X_test.shape))
print("Test set shape (y) {}".format(y_test.shape))

RNN Modeli oluşturalım

In [ ]:
input_shape= X_train.shape[1]

model= Sequential()
model.add(Dense(32, activation="relu", input_shape=(input_shape,), name="first_layer"))
model.add(Dense(16, activation="relu", name="layer_1"))
model.add(Dense(4, activation="relu", name="layer_2"))
model.add(Dense(1, name="output"))

In [ ]:
#Compiling
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#Fitting
callback= keras.callbacks.EarlyStopping(monitor="loss", patience=3)
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=2, validation_split=0.2, callbacks=[callback])

In [ ]:
train_pred= model.predict(X_train)
test_pred= model.predict(X_test)

plt.figure(figsize=(14,7))
ax1= plt.scatter(y_train, train_pred, alpha=0.7, color="green")
ax2= plt.scatter(y_test, test_pred, alpha=0.9, color="purple")
ax3= plt.plot(y_test, y_test, color="darkblue")
plt.title("Real And Prediction Values", fontdict=title_font)
plt.xlabel("Real Values", fontdict=label_font)
plt.ylabel("Prediction Values",fontdict=label_font)
plt.legend([ax1,ax2], ["Train Set", "Test Set"])
plt.show()

Modelimizin doğruluk skoruna göre tahmin performansını değerlendirelim.

In [ ]:
X_train.info()

In [ ]:
train_evaluate=model.evaluate(X_train, y_train)
test_evaluate=model.evaluate(X_test, y_test)
print('accuracy for Train set is',train_evaluate)
print('accuracy for Test set is',test_evaluate)# evaluation of model.

In [ ]:
X_train.head()

In [ ]:
stroke.head()

ANN Algoritması ile deneyelim.

In [ ]:
import tensorflow as tf

In [ ]:
model='ANN'
print("Running-------->",model)
# Part 2 - Building the ANN
# Initializing the ANN
ann = tf.keras.models.Sequential()
# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=120,input_dim=X_train.shape[1], activation='relu'))
# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=100, activation='sigmoid'))
# activation='tanh'
# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Part 3 - Training the ANN
# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [ ]:
ann.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size = 100, epochs = 10, verbose=2)

In [ ]:
y_score = ann.predict(X_test)
print (y_score)

In [ ]:
y_pred = (y_score >= 0.5).astype(int)
print(y_pred)